In [136]:
import keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional, Dropout
from keras.optimizers import Adam
import joblib

USER_ID = 353
PATH_DATA = '../data/user_sequences/'
EXTENSION_TEXT = '.txt'
PATH_USER_DATA = PATH_DATA + f'user_{USER_ID}_sequence' + EXTENSION_TEXT

data = pd.read_csv(PATH_USER_DATA, header=None)
data = data.to_numpy()[:,1:]  # Discard weekday, just for now
print(data[0:3,:])

[[38.7364443 -9.1407651]
 [38.7356124 -9.1396516]
 [38.7368552 -9.1396499]]


In [137]:
print(data.shape)
#X = data[0:-1,:]
#y = data[1:,:]

X = data
y = data

"""
#scaler = StandardScaler()
scaler = MinMaxScaler()
scaler.fit(X)
X = scaler.transform(X)
y = scaler.transform(y)
"""

print(X)

#joblib.dump(scaler, filename='..\\models\\test\\scaler')

(240, 2)
[[38.7364443 -9.1407651]
 [38.7356124 -9.1396516]
 [38.7368552 -9.1396499]
 [38.7366859 -9.1401288]
 [38.7426    -9.1281915]
 [38.7626552 -9.1134559]
 [38.762581  -9.113471 ]
 [38.7659005 -9.112833 ]
 [38.762581  -9.113471 ]
 [38.7656758 -9.1126898]
 [38.736365  -9.1408112]
 [38.7363513 -9.1408527]
 [38.7365465 -9.1395829]
 [38.7353711 -9.1440606]
 [38.7351449 -9.1439804]
 [38.7684558 -9.1121203]
 [38.7658764 -9.1127103]
 [38.7678776 -9.0995182]
 [38.7356687 -9.1417988]
 [38.7363622 -9.1408439]
 [38.7363154 -9.1372464]
 [38.7362907 -9.1407582]
 [38.7363901 -9.140801 ]
 [38.7398998 -9.1232738]
 [38.7623558 -9.113963 ]
 [38.7658914 -9.1127899]
 [38.7608908 -9.1118057]
 [38.7366267 -9.1402762]
 [38.7364037 -9.1407731]
 [38.7398699 -9.123292 ]
 [38.7659076 -9.1126593]
 [38.7625811 -9.1134718]
 [38.7625857 -9.1134708]
 [38.7626052 -9.1134665]
 [38.7658339 -9.1128388]
 [38.7422059 -9.1204622]
 [38.736473  -9.1406918]
 [38.7363762 -9.1407695]
 [38.7657867 -9.1129263]
 [38.7549619 -9.

In [138]:
kf = KFold(n_splits=8)
sequence_length = 3
epochs = 50
batch_size = 4

best_score = 10000
best_model = None
best_test_index = None

for train_index, test_index in kf.split(X):
    X_train = X[train_index, :]
    y_train = y[train_index, :]
    X_test = X[test_index, :]
    y_test = y[test_index, :]

    train_generator = TimeseriesGenerator(X_train, y_train, length=sequence_length, batch_size=batch_size, shuffle=True)
    test_generator = TimeseriesGenerator(X_test, y_test, length=sequence_length, batch_size=batch_size, shuffle=True)

    optimizer = Adam(learning_rate=0.0001) #Default = 0.001

    lstm_model = Sequential()
    lstm_model.add(LSTM(10, input_shape=(sequence_length, 2), return_sequences=True, activation=None))
    #lstm_model.add(Dropout(0.2))
    lstm_model.add(LSTM(10, return_sequences=False, activation=None))
    lstm_model.add(Dense(2, activation=None))
    lstm_model.compile(optimizer=optimizer, loss='huber')

    lstm_history = lstm_model.fit(train_generator, epochs=epochs, verbose=1, batch_size=batch_size)

    lstm_score = lstm_model.evaluate(test_generator, verbose=0, batch_size=batch_size)

    print(f'LSTM SCORE: {lstm_score}')

    if lstm_score < best_score:
        best_score = lstm_score
        best_model = lstm_model
        best_test_index = test_index


52/52 [==============================] - 0s 3ms/step - loss: 0.0015
Epoch 19/50
52/52 [==============================] - 0s 3ms/step - loss: 9.6271e-04
Epoch 20/50
52/52 [==============================] - 0s 3ms/step - loss: 9.6352e-04
Epoch 21/50
52/52 [==============================] - 0s 3ms/step - loss: 0.0012
Epoch 22/50
52/52 [==============================] - 0s 3ms/step - loss: 0.0016
Epoch 23/50
52/52 [==============================] - 0s 3ms/step - loss: 0.0012
Epoch 24/50
52/52 [==============================] - 0s 2ms/step - loss: 0.0016
Epoch 25/50
52/52 [==============================] - 0s 2ms/step - loss: 0.0011
Epoch 26/50
52/52 [==============================] - 0s 2ms/step - loss: 9.9265e-04
Epoch 27/50
52/52 [==============================] - 0s 2ms/step - loss: 0.0015
Epoch 28/50
52/52 [==============================] - 0s 2ms/step - loss: 0.0012
Epoch 29/50
52/52 [==============================] - 0s 2ms/step - loss: 0.0011
Epoch 30/50
52/52 [=====================

In [140]:
val = 5

print(f'Best score: {best_score}')
print(f'Test Index: {best_test_index}')

pred_X = np.expand_dims(X[val:val + sequence_length, :], 0)
pred = best_model.predict(pred_X)
print(f'Predicted: {pred}')
print(f'True: {X[val + sequence_length + 1,:]}')

best_model.save('..\\models\\test\\model.h5')

Best score: 0.0001127175273722969
Test Index: [30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]
1/1 [==============================] - 0s 20ms/step
Predicted: [[38.746067 -9.118415]]
True: [38.7656758 -9.1126898]
